<a href="https://colab.research.google.com/github/michalcohen/DataAnalysisABC/blob/main/find_invariant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installs
!pip install jupyter-dash
!pip install dash-cytoscape

In [7]:
# imports
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash
import dash_cytoscape as cyto
import urllib.request, json
import statistics

In [ ]:
# Load Data
df = px.data.tips()
with urllib.request.urlopen("https://raw.githubusercontent.com/michalcohen/DataAnalysisABC/main/pass1.json") as url:
    data = json.loads(url.read().decode())

In [ ]:
graph = {node["data"]["id"]: {"neighbors": list(), "inv": node["classes"] == "inv"} for node in data["nodes"]}
for edge in data["edges"]:
  graph[edge["data"]["source"]]["neighbors"].append(edge["data"]["target"])
degrees = []
for node_id, node_content in graph.items():
  node_content['degree'] = len(node_content["neighbors"])
  degrees.append(node_content['degree'])
std_deg, mean_deg, median_deg, max_deg = statistics.stdev(degrees), statistics.mean(degrees), statistics.median(degrees), max(degrees)
high_deg_nodes = [node_id for node_id, node_content in graph.items() if node_content['degree'] > mean_deg + std_deg]
high_deg_nodes

In [14]:
invariants = [node_id for node_id, node_content in graph.items() if node_content["inv"]]
len(invariants) / len(graph)

0.22560537041476864

In [9]:
# Build App
app = JupyterDash(__name__)
elements = data["nodes"] + data["edges"]
app.layout = html.Div([
                       html.P("Dash Cytoscape:"),
                       cyto.Cytoscape(id='cytoscape',
                                      elements=elements,
                                      layout={'name': 'breadthfirst'},
                                      style={'width': '800px', 'height': '1000px'},
                                      stylesheet = [
        # Group selectors
        {
            'selector': 'node',
            'style': {
                'content': 'data(label)'
            }
        },

        # Class selectors
        {
            'selector': '.inv',
            'style': {
                'background-color': 'red',
                'line-color': 'red'
            }
        },
        {
            'selector': '.non_inv',
            'style': {
                'background-color': 'blue',
                'line-color': 'blue'
            }
        }
    ]
                                      )
                       ])
app.run_server(mode='inline')

<IPython.core.display.Javascript object>